# `operator`

The operator is a fast LLM-powered agent that delegates tasks to actors. Actors
can be other LLM agents, or a function. The operator has the following
capabilities:

operator:

-   has short term memory (runtime state)
-   can decide to take multiple actions in sequence (if the actions have
    interdependencies)
-   can decide to take multiple actions in parallel (if the actions are
    independent)
-   can reflect on its own actions and take further actions based on the
    results of previous actions
-   decides when to stop taking actions (when the task is complete)

actors:

-   can be other LLM agents
-   can be functions
-   can be other operators
-   can be highly specified (e.g., json formatting)

## Example Case: Search

-   operator: user submits a query
    -   determination of intent = search
    -   actions to take:
        -   use search agent
            -   search agent
                -   search for chunks
                -   get three top results
                -   return to operator
        -   verify results
            -   results are relevant to query -> eloquently return results
            -   results are not relevant to query -> ask user to rephrase query

## Plugin Architecture

-   plugin
    -   tests
    -   registration
        -   tool/function is registered with operator to know what the tool does
        -   required inputs
        -   expected outputs

# Tools

Tools are defined as plugins. They are registered with the operator with the following components:

-   name
-   description
-   type
    -   regular function
    -   LLM function
    -   operator (has child tools)
    -   ...
-   inputs
    -   plain text or POJO
    -   can be yaml, json, or other
-   outputs
    -   plain text or POJO
    -   can be yaml, json, or other
-   tests
    -   test 1
        -   input
        -   expected output
    -   test 2
        -   input
        -   expected output

## Translator/Argument Formatter

-   formats yaml (text) ML outputs (given a text syntax spec) into json
-   returns json as POJO (not string)

### Example

#### Input

```yaml
one:
    - id: 1
      name: franc
    - id: 11
      name: Tom

two:
    - something
    - something else

three: another thing
```

#### Output

```json
{
    "one": [
        {
            "id": 1,
            "name": "franc"
        },
        {
            "id": 11,
            "name": "Tom"
        }
    ],
    "two": ["something", "something else"],
    "three": "another thing"
}
```

#### Libs

-   js: https://www.npmjs.com/package/js-yaml
-   python: https://pyyaml.org/wiki/PyYAML


In [1]:
import sys
sys.path.append("../")
from fns.openai_fns import messages_prompt, summarize

summary = summarize("""
Once upon a midnight dreary, while I pondered, weak and weary, over many a
quaint and curious volume of forgotten lore, while I nodded, nearly napping,
suddenly there came a tapping, as of some one gently rapping, rapping at my
chamber door. "'Tis some visitor," I muttered, "tapping at my chamber door— only
this and nothing more." Ah, distinctly I remember it was in the bleak December;
and each separate dying ember wrought its ghost upon the floor. Eagerly I wished
the morrow;—vainly I had sought to borrow from my books surcease of sorrow—
sorrow for the lost Lenore— for the rare and radiant maiden whom the angels name
Lenore— nameless here for evermore. And the silken, sad, uncertain rustling of
each purple curtain thrilled me—filled me with fantastic terrors never felt
before; so that now, to still the beating of my heart, I stood repeating "'Tis
some visitor entreating entrance at my chamber door— some late visitor
entreating entrance at my chamber door;— this it is and nothing more." Presently
my soul grew stronger; hesitating then no longer, "Sir," said I, "or Madam, truly
your forgiveness I implore; but the fact is I was napping, and so gently you
came rapping, and so faintly you came tapping, tapping at my chamber door, that
I scarce was sure I heard you"—here I opened wide the door;— Darkness there and
nothing more. Deep into that darkness peering, long I stood there wondering,
fearing, doubting, dreaming dreams no mortal ever dared to dream before; but the
silence was unbroken, and the stillness gave no token, and the only word there
spoken was the whispered word, "Lenore?" This I whispered, and an echo murmured
back the word, "Lenore!"— Merely this and nothing more. Back into the chamber
turning, all my soul within me burning, soon again I heard a tapping somewhat
louder than before. "Surely," said I, "surely that is something at my window
lattice; let me see, then, what thereat is, and this mystery explore— let my
heart be still a moment and this mystery explore;— 'Tis the wind and nothing
more!" Open here I flung the shutter, when, with many a flirt and flutter, in
there stepped a stately Raven of the saintly days of yore; not the least
obeisance made he; not a minute stopped or stayed he; but, with mien of lord or
lady, perched above my chamber door— perched upon a bust of Pallas just above my
chamber door— perched, and sat, and nothing more. Then this ebony bird beguiling
my sad fancy into smiling, by the grave and stern decorum of the countenance it
wore, "Though thy crest be shorn and shaven, thou," I said, "art sure no craven,
ghastly grim and ancient Raven wandering from the Nightly shore— tell me what thy
lordly name is on the Night's Plutonian shore!" Quoth the Raven "Nevermore." Much
I marvelled this ungainly fowl to hear discourse so plainly, though its answer
little meaning—little relevancy bore; for we cannot help agreeing that no
living human being ever yet was blessed with seeing bird above his chamber door—
          
""")

print(summary)

# Summary

The narrator, while reading forgotten lore late at night, hears a tapping at his door. He initially dismisses it as a visitor but finds nothing upon opening the door. The narrator is filled with sorrow for the lost Lenore and is startled by the rustling of curtains. He reassures himself that it's just a late visitor seeking entrance. Upon opening the door again, he finds only darkness and whispers "Lenore?" into it, receiving an echo in response.

His soul burning with curiosity, he hears another tapping louder than before and assumes it's something at his window lattice. Upon opening the shutter, a stately Raven from ancient times enters and perches above his chamber door on a bust of Pallas. The Raven's stern decorum amuses him and he asks its name to which it replies "Nevermore". The narrator marvels at this bird speaking so plainly despite its answer bearing little relevance.

**Keywords**: Midnight dreary, weak and weary, quaint curious volume of forgotten lore, tapping

In [2]:
import json

summary = """
- The narrator is alone in their chamber, feeling weak and weary
- They hear a tapping at their chamber door and assume it is a visitor
- The narrator is filled with sorrow for the lost Lenore
- They open the door to find darkness and silence
- The narrator hears a tapping at their window lattice and assumes it is just the wind
- They open the shutter to find a stately Raven perched above their chamber door
- The Raven speaks, saying "Nevermore"
- The narrator marvels at the Raven's ability to speak but finds its answer meaningless
"""

available_roles = {
    "system": "system",
    "user": "user",
    "assistant": "assistant",
}

text_to_json_spec = {
    "name": "yaml_to_json",
    "description": "converts yaml text to a json object",
    "arguments": {
        "yaml": "the yaml text to convert",
    }
}

elloquence_spec = {
    "name": "elloquence",
    "description": "takes a chunk of text and rephrases it to directly address the user's query",
    "arguments": {
        "text": "the text to rephrase",
    }
}

default = {
    "name": "default",
    "description": "the default tool, used when no other options seem viable",
    "arguments": {
        "text": "the text to return"
    }
}

specs = [text_to_json_spec, elloquence_spec, default]


def json_to_md_table(specs=specs):
    """
    takes an array of json objects and returns a markdown table as plain text
    """
    table = "\n| tool name | tool description |\n| --- | --- |\n"
    for spec in specs:
        table += f"| {spec['name']} | {spec['description']} |\n"
    return table


def system_prompt(specs=specs):
    """
    prompt composed of the system's available roles
    """
    return f"""
You are an assistant that has a number of actions you can take to help the user.
The tools you have available are:

{json.dumps(specs, indent=4)}

Your goal is to decide which tool to use to help the user. If multiple tools are
needed to help the user, you can use multiple tools. If multiple tools are
needed, return them in a list ordered by priority. If no tools are needed,
return an empty list. If you are unsure which tool to use, return the default
tool.
 
Return only the tool name and the inputs (payload and type) needed for the tool
as a list. No additional context is needed.

The "eloquence" tool should always be added at the end of the list if it is not
already present.

"""


system = system_prompt(specs)

print(system)

response = messages_prompt([
    {
        "role": "system",
        "content": system
    },
    {
        "role": "user",
        "content": f"Please help me make a summary from the following text {summary}"
    },
    {
        "role": "assistant",
        "content": ""
    }
])

print(response)


You are an assistant that has a number of actions you can take to help the user.
The tools you have available are:

[
    {
        "name": "yaml_to_json",
        "description": "converts yaml text to a json object",
        "arguments": {
            "yaml": "the yaml text to convert"
        }
    },
    {
        "name": "elloquence",
        "description": "takes a chunk of text and rephrases it to directly address the user's query",
        "arguments": {
            "text": "the text to rephrase"
        }
    },
    {
        "name": "default",
        "description": "the default tool, used when no other options seem viable",
        "arguments": {
            "text": "the text to return"
        }
    }
]

Your goal is to decide which tool to use to help the user. If multiple tools are
needed to help the user, you can use multiple tools. If multiple tools are
needed, return them in a list ordered by priority. If no tools are needed,
return an empty list. If you are unsure whi

In [3]:
load = json.loads(response)

load

[{'name': 'elloquence',
  'arguments': {'text': '- The narrator is alone in their chamber, feeling weak and weary\n- They hear a tapping at their chamber door and assume it is a visitor\n- The narrator is filled with sorrow for the lost Lenore\n- They open the door to find darkness and silence\n- The narrator hears a tapping at their window lattice and assumes it is just the wind\n- They open the shutter to find a stately Raven perched above their chamber door\n- The Raven speaks, saying "Nevermore"\n- The narrator marvels at the Raven\'s ability to speak but finds its answer meaningless'}}]

In [4]:
sequencer_prompt = """
You are an assistant that has a number of actions you can take to help the user.
Your job is to decide which action(s) to take and in what order. If multiple
actions are needed, return them in a list ordered by priority. If no actions are
needed, return an empty list. If you are unsure which action to take, return the
default action.

Actions Available:
    - action: web_search
      query: the query to search for
    - action: think
      thought: think about the correct action(s) to take and return them
    - action: eloquence
      text: rephrase a chunk of text to directly address the user's query
    - action: default
      text: reply politely with 'I am sorry, I do not understand'
    - action: summarize
      text: summarize a chunk of text

Example:

User: "Please find the best quote for me about living life to the fullest"

Assistant:
steps:
    - action: web_search
      query: quote about living life to the fullest
    - action: think
      thought: from the the list of results, find the one that is most relevant to the user's query
    - action: eloquence
      text: the text from the most relevant result
"""
response = messages_prompt([
    {
        "role": "system",
        "content": sequencer_prompt
    },
    {
        "role": "user",
        "content": f"How much wood can a beaver cut down in a day?"
    },
    {
        "role": "assistant",
        "content": ""
    }
])

"""
When pondering on a happy life, Charles Darwin once said: 
With regards to living a full life, Charles Darwin said: 'A man who dares to waste one hour of time has not discovered the value of life.'
"""

print(response)

steps:
    - action: web_search
      query: how much wood can a beaver cut down in a day


## Outline

1. user submits query
2. operator determines intent
3. operator determines first action to take
4. operator delegates action to actor
5. actor performs action
6. actor returns results to operator
7. operator determines next action to take or that the task is complete
   a. if task is complete, operator returns results to user
   b. if task is not complete, repeat from step 3


In [5]:
import json
import os
from typing import List

import requests
from dotenv import load_dotenv
from markdownify import markdownify as md

load_dotenv(dotenv_path="../.env")

API_KEY = os.getenv("BRAVE_SEARCH_API_KEY")


def web_search(
    query: str,
    search_kwargs: dict = {},
    api_key: str = API_KEY
):
    """Wrapper around the Brave search engine."""

    base_url: str = "https://api.search.brave.com/res/v1/web/search"
    """The base URL for the Brave search engine."""

    def search_request(query: str) -> List[dict]:
        headers = {
            "X-Subscription-Token": api_key,
            "Accept": "application/json",
        }
        req = requests.PreparedRequest()
        params = {**search_kwargs, **{"q": query}}
        req.prepare_url(base_url, params)
        if req.url is None:
            raise ValueError("prepared url is None, this should not happen")

        response = requests.get(req.url, headers=headers)
        if not response.ok:
            raise Exception(f"HTTP error {response.status_code}")

        return response.json().get("web", {}).get("results", [])

    web_search_results = search_request(query)

    final_results = [
        {
            "title": item.get("title"),
            "url": item.get("url"),
            "snippet": md(item.get("description")),
        }
        for item in web_search_results
    ]

    dump = json.dumps(final_results)
    return dump


# test
results = web_search("how much wood can a beaver cut down in a day")

# pretty print json results
print(json.dumps(json.loads(results), indent=4))

[
    {
        "title": "4 Reasons Why Beavers Cut Down Trees (And How They Do It) - Tree Journey",
        "url": "https://treejourney.com/reasons-why-beavers-cut-down-trees-and-how-they-do-it/",
        "snippet": "Beavers can cut down as many as 200 trees a year, but what does their rate of chewing seriously look like? One beaver can remove **nearly 150 chips** of wood from a tree with a 5-inch diameter, which would topple the tree in mere moments. This means it would take about 8 minutes to cut down a ..."
    },
    {
        "title": "How many trees can a beaver cut down in 24 hours? - Quora",
        "url": "https://www.quora.com/How-many-trees-can-a-beaver-cut-down-in-24-hours",
        "snippet": "Answer: How many trees **can** **a** **beaver** **cut** **down** **in** 24 hours? That depends mostly on the size of the trees. **A** **beaver** **can** gnaw through a cottonwood tree that is two feet in diameter in an hour or two. Smaller trees are easier for them to move, though, 

In [6]:
import requests
from fns.soup_fns import html2md_clean


def page_scraper(url):
    """
    scrapes the text from a web page
    """
    response = requests.get(url)
    if not response.ok:
        raise Exception(f"HTTP error {response.status_code}")
    md = html2md_clean(response.text)
    return md


# test
text = page_scraper(
    "https://animals.mom.com/long-beaver-chew-down-tree-11371.html")
print(text)

How Long Does It Take for a Beaver to Chew Down a Tree? | Pets on Mom.com      Our Privacy/Cookie Policy contains detailed information about the types of cookies & related technology on our site, and some ways to opt out. By using the site, you agree to the uses of cookies and other technology as outlined in our Policy, and to our Terms of Use. Close

        * Cats
* Dogs
* Small Pets
* Other Animals

 * 
* 
* 
* 

      * Birds•
* Farm Animals•
* Fish•
* Insects•
* Wildlife and Exotic Animals•
* Reptiles, Rodents and Small Animals•
* House Pets

         # How Long Does It Take for a Beaver to Chew Down a Tree?

 By Bridget Cipollini          ![](/public/images/logo-fallback.svg)   i Jupiterimages/Photos.com/Getty Images    We’ve all heard the old saying “busy as a beaver”. The fact is, beavers (Castor canadensis) really do keep busy, especially at night. In fact, beavers are so industrious, a lone beaver is capable of felling an 8-foot tree in 5 minutes.

  ## They're Expert Lumberj

In [7]:
actions = [
    {
        "name": "web_search",
        "description": "Searches the web for the user's query",
        "arguments": {
            "query": "The user's query"
        }
    },
    {
        "name": "match_maker",
        "description": "From any given list of results, returns the text from the one that is most relevant to the user's query",
        "arguments": {
            "results": "A list of results to choose from",
            "query": "The users query"
        }
    },
    {
        "name": "page_scraper",
        "description": "If more information is required and a URL is available, this tool can get more content from the page at that URL",
        "arguments": {
            "url": "The URL for the page to scrape"
        }
    },
    {
        "name": "eloquence",
        "description": "Transforms a decontextualized chunk of text into a response that directly addresses the user's query",
        "arguments": {
            "text": "The text to rephrase"
        }
    },
    {
        "name": "default",
        "description": "The default tool, used when no other options seem viable",
        "arguments": {
            "text": "The text to return"
        }
    },
    {
        "name": "end",
        "description": "When the user's query is complete, this ends the conversation and returns the final response",
        "arguments": {
            "text": "The text to return"
        }
    }
]

In [8]:
import asyncio
import json

from fns.openai_fns import messages_prompt


def match_maker(
    results,
    query,
):
    """
    from any given list of results, returns the text from the one that is most
    relevant to the user's query
    """
    prompt = """
    You are a helpful assistant that takes a list of search results and a user
    query and returns the most relevant result to the user's query.
    """
    response = messages_prompt([
        {
            "role": "system",
            "content": prompt
        },
        {
            "role": "user",
            "content": f"User Query: {query}\n\nSearch Results: {results}"
        },
    ])
    print("match_maker response: ", response)
    return response


def assistant_prompt(actions):
    return f"""
You are an assistant that has a number of actions you can take to help the user.
These actions are: {json.dumps(actions, indent=4)}

You will be provided with the user's query and the last response from the host.
Your job is to decide if the last response fully addresses the user's query. If
so, return the end action. If not, return the action that you think will get the
user closer to their goal. If you are unsure which action to take, return the
default action.

Please only return the tool name with the arguments needed to run the tool in
JSON format as per the action's JSON spec. When the user's problem is solved,
finish with the `end` action. No additional context is needed.
"""


def default(text):
    return text


toolbox = {
    "web_search": web_search,
    "match_maker": match_maker,
    "page_scraper": page_scraper,
    "default": default,
    "end": default
}


def agent_assistant(
    user_query,
    latest_info,
    actions=actions
):
    """
    This agent decides what should be done next. It has a selection of actions
    that it can decide to use if appropriate to the current context.

    If the agent decides to use a tool, it should return the tool name and the
    arguments needed to run the tool in JSON format as per the spec below:
    """
    system_prompt = assistant_prompt(actions)
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_query
        }
    ]
    if latest_info != "":
        # print("👓 agent_assistant > latest_info: ", latest_info)
        messages.append(latest_info)
    else:
        print("\n👓 agent_assistant > latest_info is empty\n")

    print(f"\n👓 agent_assistant > messages:\n {messages}\n")
    response = messages_prompt(messages)
    print(f"\n👓 agent_assitant response:\n{response}\n")
    try:
        load = json.loads(response)
        print(f"👓 agent_assistant load:\n{json.dumps(load, indent=2)}\n")
        tool_name = load["name"]
        tool_args = load["arguments"]
        tool = toolbox[tool_name]
        message = tool(**tool_args)
        return {
            "name": tool_name,
            "arguments": tool_args,
            "message": message
        }
    except Exception as e:
        print(e)
        return "I am sorry, I'm having trouble with actions right now"


test_history = [
    {
        "role": "system",
        "content": "this is a prompt"
    },
    {
        "role": "user",
        "content": f"How much wood can a beaver cut down in a day?"
    },
    {
        "role": "assistant",
        "content": "your wish is my command"
    },
    {
        "role": "system",
        "content": "you need to do a search"
    },
    {
        "role": "assistant",
        "content": "I searched for you"
    }
]


def agent_host(
    history=[]  # chat history
):
    """
    This agent decides if the last fully addresses the user query.
    If so, it returns "end"
    If not, it returns "continue"
    """
    # find the first item in the history list with the role "user"
    user_query = next(
        (item for item in history if item["role"] == "user"),
        None
    )
    query_content = user_query["content"]
    # find the last item in the history list with the role "assistant"
    latest_info = next(
        (item for item in reversed(history) if item["role"] == "assistant"),
        None
    )
    if latest_info is None:
        latest_info = ""
    # send the query and the last response to the assistant to determine if it's satisfied
    action = agent_assistant(query_content, latest_info)
    print("💁‍♂️ agent_host action: ", action)
    return action


# test
# agent_host(test_history)


shorterm_memory = {
    "query_statisfied": False,
    "all_messages": [],
    "relavant_messages": []
}

queries = [
    {
        "query": "Add a taxpayer to my account",
        "sequence": actions
    }
]


async def recursive_awaiter(
    history=[{"role": "system", "content": "this is a prompt"}]
):
    """
    delegates user_query to the agent host. Once the agent host returns "end",
    the function returns the last message to the user.
    """
    action = agent_host(history)
    print("🤔 recursive_awaiter action: ", action)
    # if type of action is dict
    if isinstance(action, dict):
        if action["name"] == "end" or action["name"] == "default":
            return action["arguments"]["text"]
        else:
            print("🤔 recursive_awaiter terminated: ", action)
            history.append({"role": "assistant", "content": action["message"]})
            return await recursive_awaiter(history)
    else:
        return action

# test
test = await recursive_awaiter(
    history=[{
        "role": "user",
        "content": "How much wood can a beaver cut down in a day?"
    }]
)

print(test)


👓 agent_assistant > latest_info is empty


👓 agent_assistant > messages:
 [{'role': 'system', 'content': '\nYou are an assistant that has a number of actions you can take to help the user.\nThese actions are: [\n    {\n        "name": "web_search",\n        "description": "Searches the web for the user\'s query",\n        "arguments": {\n            "query": "The user\'s query"\n        }\n    },\n    {\n        "name": "match_maker",\n        "description": "From any given list of results, returns the text from the one that is most relevant to the user\'s query",\n        "arguments": {\n            "results": "A list of results to choose from",\n            "query": "The users query"\n        }\n    },\n    {\n        "name": "page_scraper",\n        "description": "If more information is required and a URL is available, this tool can get more content from the page at that URL",\n        "arguments": {\n            "url": "The URL for the page to scrape"\n        }\n    },\n    {\n 

```mermaid
graph TD;
    A[User] --> B[Agent]
    B --> C[Action]
    C --> B
```
